<a href="https://colab.research.google.com/github/iko-ai/ibibio-tts/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os
from glob import glob
import csv
import random
import pandas as pd

In [3]:
print(len(os.listdir("/content/drive/MyDrive/dataset/wav")))

604


In [ ]:
# Paths
input_excel_path = '/content/metadata.xlsx'
output_csv_path = '/content/drive/MyDrive/dataset/metadata.csv'

# Read the Excel file
df = pd.read_excel(input_excel_path, engine='openpyxl')

# Save the dataframe as a CSV file
df.to_csv(output_csv_path, index=False, encoding='utf-8')

print("Excel file has been successfully converted to CSV.")


Excel file has been successfully converted to CSV.


In [ ]:
# Paths
input_csv_path = '/content/drive/MyDrive/dataset/metadata.csv'
input_wavs_dir = '/content/drive/MyDrive/dataset/wav'
output_dir = '/content/drive/MyDrive/dataset/wav_split'
train_txt_path = os.path.join(output_dir, 'train.txt')
val_txt_path = os.path.join(output_dir, 'val.txt')

In [ ]:
# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Read the input CSV
with open(input_csv_path, 'r') as csv_file:
    reader = csv.DictReader(csv_file)
    data = list(reader)

# Shuffle the data
random.shuffle(data)

# Split the data into 80% training and 20% validation
split_index = int(0.8 * len(data))
train_data = data[:split_index]
val_data = data[split_index:]

# Function to write data to a text file
def write_data(file_path, data):
    with open(file_path, 'w') as file:
        for row in data:
            filename = row['Filepath']
            transcription = row['Transcription']
            file.write(f"wavs/{filename}|{transcription}\n")


In [ ]:
# Write the training and validation data to their respective files
write_data(train_txt_path, train_data)
write_data(val_txt_path, val_data)

print("Dataset has been successfully split and saved.")

Dataset has been successfully split and saved.


In [4]:
!git clone https://github.com/iko-ai/ibibio-tts.git

Cloning into 'ibibio-tts'...
remote: Enumerating objects: 16650, done.
remote: Counting objects: 100% (419/419), done.
remote: Compressing objects: 100% (212/212), done.
remote: Total 16650 (delta 232), reused 358 (delta 206), pack-reused 16231
Receiving objects: 100% (16650/16650), 12.16 MiB | 23.58 MiB/s, done.
Resolving deltas: 100% (13345/13345), done.


In [7]:
%cd ../

/content


In [8]:
#%cd ibibio-texttospeech/codes
#%cd codes
%cd /content/ibibio-tts/codes

/content/ibibio-tts/codes


In [ ]:
!pip install -r requirements.laxed.txt

experiments/dvae.pth: No such file or directory
experiments/autoregressive.pth: No such file or directory


In [4]:
!wget https://huggingface.co/Gatozu35/tortoise-tts/resolve/main/dvae.pth -O /content/ibibio-tts/experiments/dvae.pth
!wget https://huggingface.co/jbetker/tortoise-tts-v2/resolve/main/.models/autoregressive.pth -O /content/ibibio-tts/experiments/autoregressive.pth

--2024-05-30 06:25:34--  https://huggingface.co/Gatozu35/tortoise-tts/resolve/main/dvae.pth
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.24, 18.172.134.88, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/9f/83/9f8300e5ccd418d283e72c07d1851f269dbd2ca7bae49cee21285965d9bebe14/a990825371506c16bcf0e8167bf24ccf82f65bb6a1dbcbfcf058d76f9b197e35?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27dvae.pth%3B+filename%3D%22dvae.pth%22%3B&Expires=1717309534&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNzMwOTUzNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy85Zi84My85ZjgzMDBlNWNjZDQxOGQyODNlNzJjMDdkMTg1MWYyNjlkYmQyY2E3YmFlNDljZWUyMTI4NTk2NWQ5YmViZTE0L2E5OTA4MjUzNzE1MDZjMTZiY2YwZTgxNjdiZjI0Y2NmODJmNjViYjZhMWRiY2JmY2YwNThkNzZmOWIxOTdlMzU%7EcmVzcG9uc2UtY29udGVud

# Make sure the waveform audio has a sampling rate of 22.05kHz
Install librosa using the following command (automatically installs soundfile as well):
pip install librosa

In [6]:
import librosa
import soundfile as sf

def resample_wav_file(input_file, target_sampling_rate=22050):
    # Load audio file
    audio, sampling_rate = librosa.load(input_file, sr=None)

    # Check if the sampling rate matches the target
    if sampling_rate != target_sampling_rate:

        # Resample audio to the target sampling rate
        audio_resampled = librosa.resample(audio, orig_sr=sampling_rate, target_sr=target_sampling_rate)

        # Overwrite the input file with the resampled audio
        sf.write(input_file, audio_resampled, target_sampling_rate)

In [10]:
# Resample all audio samples to 22.05kHz
dataset_path = '/content/drive/MyDrive/dataset/wav'
for wav_file in glob(dataset_path + "*.wav"):
    resample_wav_file(wav_file)

# Fine-Tune THE AUTOREGRESSIVE MODEL

In [ ]:
!python3 /content/ibibio-tts/codes/train.py -opt /content/ibibio-tts/experiments/custom_language_gpt.yml

Disabled distributed training.
24-05-30 06:36:04.527 - INFO:   name: custom_language_gpt
  model: extensibletrainer
  scale: 1
  gpu_ids: [0]
  start_step: 0
  checkpointing_enabled: True
  fp16: False
  use_8bit: True
  wandb: False
  use_tb_logger: True
  datasets:[
    train:[
      name: train_dataset
      n_workers: 8
      batch_size: 128
      mode: paired_voice_audio
      path: /content/drive/MyDrive/dataset/wav_split/train.txt
      fetcher_mode: ['lj']
      phase: train
      max_wav_length: 255995
      max_text_length: 200
      sample_rate: 22050
      load_conditioning: True
      num_conditioning_candidates: 2
      conditioning_length: 44000
      use_bpe_tokenizer: True
      load_aligned_codes: False
      tokenizer_vocab: /content/drive/MyDrive/dataset/tokenizer.json
      data_type: img
    ]
    val:[
      name: val_dataset
      n_workers: 1
      batch_size: 64
      mode: paired_voice_audio
      path: /content/drive/MyDrive/dataset/wav_split/val.txt
      f

In [14]:
print(os.getcwd())

/content
